In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Carga Delta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

In [0]:
# Define os caminhos de armazenamento no Data Lake
silver_path = "/Volumes/workspace/lhdw/silver/vendas"
gold_path = "/Volumes/workspace/lhdw/gold/vendas_delta"

In [0]:
df_silver = spark.read.format("parquet").load(silver_path)

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
#Nome tabela destino

tb_destino = "dim_produto"

# Extrair produtos únicos para a dimensão Produto
dim_produto_df = df_silver.select(
    "IDProduto", "Produto", "Categoria").dropDuplicates()

# Adicionar chave substituta (surrogate keys)
dim_produto_df = dim_produto_df.withColumn("sk_produto", monotonically_increasing_id()+1)

# Escrever DimProduto no formato Delta
dim_produto_df.write.format("delta").mode("overwrite").save(f"{gold_path}/{tb_destino}")

#display(dim_produto_df)
#dim_produto_df.count()

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
#Nome tabela destino

tb_destino = "dim_categoria"

# Extrair Categorias únicas para a dimensão Categoria
dim_categoria_df = df_silver.select(
    "Categoria").dropDuplicates()

# Adicionar chave substituta (surrogate keys)
dim_categoria_df = dim_categoria_df.withColumn("sk_categoria", monotonically_increasing_id()+1)

# Escrever DimCatgoria no formato Parquet, particionando por Categoria
dim_categoria_df.write.format("delta").mode("overwrite").save(f"{gold_path}/{tb_destino}")

dim_categoria_df.count()

5

In [0]:
#Nome tabela destino

tb_destino = "dim_segmento"

# Extrair Segmentos únicos para a dimensão Segmentos
dim_segmento_df = df_silver.select(
   "Segmento").dropDuplicates()

# Adicionar chave substituta (surrogate keys)
dim_segmento_df = dim_segmento_df.withColumn("sk_segmento", monotonically_increasing_id()+1)

# Escrever DimSegmento no formato Parquet
dim_segmento_df.write.format("delta").mode("overwrite").save(f"{gold_path}/{tb_destino}")

dim_segmento_df.count()


9

In [0]:
#Nome tabela destino
tb_destino = "dim_fabricante"

# Extrair produtos únicos para a dimensão Fabricante    
dim_fabricante_df = df_silver.select(
    "IDFabricante", "Fabricante").dropDuplicates()

# Adicionar chave substituta (surrogate keys)
dim_fabricante_df = dim_fabricante_df.withColumn("sk_fabricante", monotonically_increasing_id()+1)

# Escrever DimFabricante no formato Delta
dim_fabricante_df.write.format("delta").mode("overwrite").save(f"{gold_path}/{tb_destino}")

dim_fabricante_df.count()

1

### Criação da Dimensão Geografia

In [0]:
#Nome tabela destino
tb_destino = "dim_geografia"

# Extrair Geografia  únicos para a dimensão Geografia
dim_geografia_df = df_silver.select(
     "Cidade", "Estado", "Regiao", "Distrito", "Pais", "CodigoPostal"
).dropDuplicates()

# Adicionar chave substituta
dim_geografia_df = dim_geografia_df.withColumn("sk_geografia", monotonically_increasing_id()+1)

# Escrever DimGeografia no formato Parquet
dim_geografia_df.write.format("delta").mode("overwrite").save(f"{gold_path}/{tb_destino}")

dim_geografia_df.count()


20467

In [0]:
#Nome tabela destino
tb_destino = "dim_cliente"

from pyspark.sql.functions import col, monotonically_increasing_id
# Passo 1 - Extrair clientes únicos para a dimensão Cliente
dim_cliente_df = df_silver.select(
    "IDCliente", "Nome", "Email", "Cidade", "Estado", "Regiao", "Distrito", "Pais", "CodigoPostal"
).dropDuplicates()

# Passo 2 - Realizar o join para obter a SK_Geografia
dim_cliente_com_sk_df = dim_cliente_df.alias("cliente") \
    .join(dim_geografia_df.alias("geografia"), 
          (col("cliente.Cidade") == col("geografia.Cidade")) &
          (col("cliente.Estado") == col("geografia.Estado")) &
          (col("cliente.Regiao") == col("geografia.Regiao")) &
          (col("cliente.Distrito") == col("geografia.Distrito")) &
          (col("cliente.Pais") == col("geografia.Pais")) &
          (col("cliente.CodigoPostal") == col("geografia.CodigoPostal")), 
          "left") \
    .select("cliente.IDCliente", "cliente.Nome", "cliente.Email", "geografia.sk_geografia")

# Passo 3 - Adicionar chave substituta
dim_cliente_com_sk_df = dim_cliente_com_sk_df.withColumn("sk_cliente", monotonically_increasing_id()+1)

# Passo 4 - Selecionar colunas específicas
dim_cliente_com_sk_df = dim_cliente_com_sk_df.select("IDCliente", "Nome","Email", "sk_geografia", "sk_cliente")

# Passo 5 - Escrever DimCliente no formato Delta
dim_cliente_com_sk_df.write.format("delta").mode("overwrite").save(f"{gold_path}/{tb_destino}")

dim_cliente_com_sk_df.count()

112081

In [0]:
#Nome tabela destino
tb_destino = "fato_vendas"

from pyspark.sql.functions import broadcast,year, month
# Juntar dados da Silver com tabelas de dimensões para obter as chaves substitutas
fato_vendas_df = df_silver.alias("s") \
    .join(broadcast(dim_produto_df.select("IDProduto", "sk_produto").alias("dprod")), "IDProduto") \
    .join(broadcast(dim_categoria_df.select("Categoria", "sk_categoria").alias("dcat")), "Categoria") \
    .join(broadcast(dim_segmento_df.select("Segmento", "sk_segmento").alias("dseg")), "Segmento") \
    .join(broadcast(dim_fabricante_df.select("Fabricante", "sk_fabricante").alias("dfab")), "Fabricante") \
    .join(broadcast(dim_cliente_com_sk_df.select("IDCliente", "sk_cliente").alias("dcli")), "IDCliente") \
    .select(
        col("s.Data").alias("DataVenda"),
        "sk_produto",
        "sk_categoria",
        "sk_segmento",
        "sk_fabricante",
        "sk_cliente",
        "Unidades",
        col("s.PrecoUnitario"),
        col("s.CustoUnitario"),
        col("s.TotalVendas")
    )

# Escrever tabela Fato no formato Delta, particionando por DataVenda (ano e mês)
fato_vendas_df.withColumn("Ano", year("DataVenda")) \
             .withColumn("Mes", month("DataVenda")) \
             .write.format("delta") \
             .mode("overwrite")\
             .option("MaxRecordsPerFile", 1000000)\
             .partitionBy("Ano", "Mes")\
             .save(f"{gold_path}/{tb_destino}")

fato_vendas_df.count()             

112202

### Limpeza de Memória

In [0]:
import gc

# Coletar lixo após operações pesadas para liberar memória
gc.collect()


3982

###Evidências de Carga na Camada Gold (Delta)

In [0]:
%fs ls /Volumes/workspace/lhdw/gold/vendas_delta/

path,name,size,modificationTime
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_categoria/,dim_categoria/,0,1756333959430
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_cliente/,dim_cliente/,0,1756333959430
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_fabricante/,dim_fabricante/,0,1756333959430
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_geografia/,dim_geografia/,0,1756333959430
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_produto/,dim_produto/,0,1756333959430
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_segmento/,dim_segmento/,0,1756333959430
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/,fato_vendas/,0,1756333959430


In [0]:
%fs ls /Volumes/workspace/lhdw/gold/vendas_delta/dim_categoria/

path,name,size,modificationTime
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_categoria/_delta_log/,_delta_log/,0,1756333971707
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_categoria/part-00000-169ce2a6-2400-411a-afd5-17782a258a29.c000.snappy.parquet,part-00000-169ce2a6-2400-411a-afd5-17782a258a29.c000.snappy.parquet,933,1756333934000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_categoria/part-00000-ef0a0c77-92f7-4ee6-b46c-d8f96157cc1d.c000.snappy.parquet,part-00000-ef0a0c77-92f7-4ee6-b46c-d8f96157cc1d.c000.snappy.parquet,933,1756155572000
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/dim_categoria/part-00000-fa8058ea-0648-4819-b2de-53d62c741fe7.c000.snappy.parquet,part-00000-fa8058ea-0648-4819-b2de-53d62c741fe7.c000.snappy.parquet,933,1756320794000


In [0]:
%fs ls /Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/Ano=2011/

path,name,size,modificationTime
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/Ano=2011/Mes=1/,Mes=1/,0,1756333973528
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/Ano=2011/Mes=10/,Mes=10/,0,1756333973528
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/Ano=2011/Mes=11/,Mes=11/,0,1756333973528
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/Ano=2011/Mes=12/,Mes=12/,0,1756333973528
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/Ano=2011/Mes=2/,Mes=2/,0,1756333973528
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/Ano=2011/Mes=3/,Mes=3/,0,1756333973528
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/Ano=2011/Mes=4/,Mes=4/,0,1756333973528
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/Ano=2011/Mes=5/,Mes=5/,0,1756333973528
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/Ano=2011/Mes=6/,Mes=6/,0,1756333973528
dbfs:/Volumes/workspace/lhdw/gold/vendas_delta/fato_vendas/Ano=2011/Mes=7/,Mes=7/,0,1756333973528
